Nama: : Jessica Theresia<br>
Email: : me.jessicatheresia@gmail.com<br>
ID Dicoding: : jessica_trs<br>

# **1. Import Library**

Tahap untuk mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV di google drive ke dalam variabel DataFrame.

In [2]:
# Mount Google Drive
drive.mount('/content/drive')

# Memuat dataset
df = pd.read_csv("/content/drive/MyDrive/Dicoding ML Submission/Belajar Machine Learning untuk Pemula/Dataset_inisiasi.csv")

# Menampilkan 5 baris pertama
print(df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   TransactionAmount TransactionType   Location DeviceID MerchantID Channel  \
0              14.09           Debit  San Diego  D000380       M015     ATM   
1             376.24           Debit    Houston  D000051       M052     ATM   
2             126.29           Debit       Mesa  D000235       M009  Online   
3             184.50           Debit    Raleigh  D000187       M002  Online   
4              13.45          Credit    Atlanta  D000308       M091  Online   

   CustomerAge CustomerOccupation  TransactionDuration  LoginAttempts  \
0         70.0             Doctor                 81.0            1.0   
1         68.0             Doctor                141.0            1.0   
2         19.0            Student                 56.0            1.0   
3         26.0            Student                 25.0            1.0   
4         26.0            Stude

## Informasi Dataset

untuk memastikan bahwa dataset sudah dilakukan clustering.

In [3]:
# Menampilkan informasi dasar dataset
dataset_info = df.info()
dataset_head = df.head()
missing_values = df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2512 entries, 0 to 2511
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TransactionAmount    2512 non-null   float64
 1   TransactionType      2512 non-null   object 
 2   Location             2512 non-null   object 
 3   DeviceID             2512 non-null   object 
 4   MerchantID           2512 non-null   object 
 5   Channel              2512 non-null   object 
 6   CustomerAge          2512 non-null   float64
 7   CustomerOccupation   2512 non-null   object 
 8   TransactionDuration  2512 non-null   float64
 9   LoginAttempts        2512 non-null   float64
 10  AccountBalance       2512 non-null   float64
 11  Cluster_KMeans       2512 non-null   int64  
dtypes: float64(5), int64(1), object(6)
memory usage: 235.6+ KB


In [4]:
dataset_head

,TransactionAmount,TransactionType,Location,DeviceID,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,Cluster_KMeans
0,14.09,Debit,San Diego,D000380,M015,ATM,70.0,Doctor,81.0,1.0,5112.21,1
1,376.24,Debit,Houston,D000051,M052,ATM,68.0,Doctor,141.0,1.0,13758.91,0
2,126.29,Debit,Mesa,D000235,M009,Online,19.0,Student,56.0,1.0,1122.35,0
3,184.50,Debit,Raleigh,D000187,M002,Online,26.0,Student,25.0,1.0,8569.06,0
4,13.45,Credit,Atlanta,D000308,M091,Online,26.0,Student,198.0,1.0,7429.40,0


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [5]:
# 1. Memisahkan fitur (X) dan target (y)
X = df.drop("Cluster_KMeans", axis=1)  # Semua kolom kecuali target
y = df["Cluster_KMeans"]               # Target label

# 2. Encoding fitur kategorikal (One-Hot Encoding)
X = pd.get_dummies(X, drop_first=True)

# 3. Membagi dataset menjadi training (80%) dan testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Menampilkan hasil pembagian data
X_train.shape, X_test.shape, y_train.value_counts(), y_test.value_counts()

((2009, 832),
 (503, 832),
 Cluster_KMeans
 0    1010
 1     999
 Name: count, dtype: int64,
 Cluster_KMeans
 0    253
 1    250
 Name: count, dtype: int64)

### Penjelasan Output

* (2009, 832): Training Set memiliki 2009 sampel (baris) dan 832 fitur (kolom) setelah dilakukan one-hot encoding pada fitur kategorikal.

* (503, 832): Testing Set memiliki 503 sampel (baris) dan 832 fitur (kolom) dengan struktur yang sama seperti training set.

* Distribusi Label di Training Set: 1010 sampel termasuk dalam kelas 1 dan 999 sampel termasuk dalam kelas 0. Seimbang (hampir 50:50), yang bagus untuk mencegah bias terhadap salah satu kelas.

* Distribusi Label di Testing Set: 253 sampel termasuk dalam kelas 1 dan 250 sampel termasuk dalam kelas 0. Distribusi di testing set juga seimbang, memastikan evaluasi yang adil untuk kedua kelas.

* **Kesimpulan**: Data telah berhasil dibagi menjadi training (80%) dan testing (20%). Distribusi label antara kelas 0 dan kelas 1 seimbang, yang baik untuk performa model klasifikasi.

# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

In [6]:
# 1. Membangun Model Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

Penjelasan

1. Random Forest
<br>
Random Forest adalah algoritma ensemble yang terdiri dari banyak Decision Tree yang dilatih secara independen menggunakan subset data dan fitur yang berbeda. Hasil prediksi diambil berdasarkan mayoritas suara (voting) untuk klasifikasi.

* Alasan Memilih Algoritma ini :

  * a. Robust terhadap Outlier dan Noise: Karena menggunakan hasil agregasi dari banyak pohon keputusan, model ini lebih tahan terhadap data yang memiliki outlier (yang sudah kita tangani sebelumnya).

  * b. Menangkap Interaksi Fitur: Dataset kita memiliki beberapa fitur numerik yang kompleks. Random Forest dapat memahami hubungan non-linear antara fitur.

  * c. Stabilitas dan Akurasi: Dengan membangun banyak pohon, model menjadi lebih stabil dan memiliki bias rendah serta varian yang moderat.

* Kaitan dengan Dataset:
  * Dataset hasil clustering mengandung variabel numerik seperti TransactionAmount dan LoginAttempts yang memiliki distribusi yang bervariasi. Random Forest efektif dalam menangkap pola kompleks dari fitur-fitur ini.* Cocok untuk dataset dengan jumlah fitur yang tidak terlalu besar dan sudah bersih setelah penanganan outlier dan fitur redundant.

In [7]:
# 2. Membangun Model Logistic Regression
lr_model = LogisticRegression(max_iter=100, random_state=42)
lr_model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

Penjelasan

2. Logistic Regression
<br>
Logistic Regression adalah metode klasifikasi yang menggunakan fungsi sigmoid untuk memodelkan probabilitas hasil biner atau multiklas.

* Alasan Memilih Logistic Regression?

  * Sederhana dan Efisien: Algoritma ini mudah diinterpretasikan dan efisien dalam komputasi meskipun dataset memiliki banyak sampel.

  * Hubungan Linear: Jika hubungan antara fitur dan target bersifat linier, Logistic Regression akan memberikan hasil yang baik.

  * Cocok untuk Dataset Terstruktur: Logistic Regression sangat baik dalam memproses dataset yang telah dibersihkan dari outlier dan redundan.

* Kaitan dengan Dataset:

  * Dataset ini memiliki label biner hasil dari proses clustering (Cluster_KMeans: 0 atau 1), yang cocok dengan fungsi sigmoid Logistic Regression.
  * Dengan fitur yang telah ditangani outlier-nya, Logistic Regression bisa memberikan hasil akurat tanpa bias dari data ekstrem.

## **b. Evaluasi Model Klasifikasi**

**Tahapan Evaluasi Model Klasifikasi yang Dibuat untuk Melakukan Prediksi**

1. Menggunakan model Random Forest dan Logistic Regression untuk memprediksi label pada data uji (X_test),

2. Menghitung Metrik Evaluasi
  * Accuracy: Persentase prediksi yang benar dari total data uji.
  * F1-Score: Rata-rata harmonik dari Precision
  * Recall untuk mengukur keseimbangan prediksi.

3. Membuat Confusion Matrix

  * Menampilkan jumlah True Positive (TP), True Negative (TN), False Positive (FP), dan False Negative (FN) untuk kedua model.

4. Laporan Klasifikasi (Classification Report)

  * Menyediakan metrik lengkap: Precision, Recall, dan F1-Score untuk masing-masing kelas.

5. Membandingkan Performa Model

  * Menyusun hasil evaluasi dalam bentuk tabel untuk membandingkan performa Random Forest dan Logistic Regression.

In [8]:
# 1. Melakukan prediksi menggunakan data uji
rf_preds = rf_model.predict(X_test)  # Prediksi Random Forest
lr_preds = lr_model.predict(X_test)  # Prediksi Logistic Regression

# 2. Metrik evaluasi (Accuracy dan F1-Score)
rf_accuracy = accuracy_score(y_test, rf_preds)
rf_f1 = f1_score(y_test, rf_preds)

lr_accuracy = accuracy_score(y_test, lr_preds)
lr_f1 = f1_score(y_test, lr_preds)

# 3. Confusion matrix
rf_conf_matrix = confusion_matrix(y_test, rf_preds)
lr_conf_matrix = confusion_matrix(y_test, lr_preds)

# 4. Precision, Recall, F1-Score
rf_report = classification_report(y_test, rf_preds)
lr_report = classification_report(y_test, lr_preds)

In [9]:
# Membuat DataFrame untuk hasil evaluasi
results = pd.DataFrame({
    "Model": ["Random Forest", "Logistic Regression"],
    "Accuracy": [rf_accuracy, lr_accuracy],
    "F1-Score": [rf_f1, lr_f1]
})

In [10]:
# Menampilkan hasil evaluasi dalam tabel
print("=== Hasil Evaluasi Model ===")
print(results.to_string(index=False))

=== Hasil Evaluasi Model ===
              Model  Accuracy  F1-Score
      Random Forest  0.691849  0.665227
Logistic Regression  0.719682  0.720792


In [11]:
# Perbandingan confusion matrix
print("\n=== Confusion Matrix - Random Forest ===")
print(rf_conf_matrix)

print("\n=== Confusion Matrix - Logistic Regression ===")
print(lr_conf_matrix)


=== Confusion Matrix - Random Forest ===
[[194  59]
 [ 96 154]]

=== Confusion Matrix - Logistic Regression ===
[[180  73]
 [ 68 182]]


In [12]:
# Perbandingan Classification Report
print("\n=== Classification Report - Random Forest ===")
print(rf_report)

print("\n=== Classification Report - Logistic Regression ===")
print(lr_report)


=== Classification Report - Random Forest ===
              precision    recall  f1-score   support

           0       0.67      0.77      0.71       253
           1       0.72      0.62      0.67       250

    accuracy                           0.69       503
   macro avg       0.70      0.69      0.69       503
weighted avg       0.70      0.69      0.69       503


=== Classification Report - Logistic Regression ===
              precision    recall  f1-score   support

           0       0.73      0.71      0.72       253
           1       0.71      0.73      0.72       250

    accuracy                           0.72       503
   macro avg       0.72      0.72      0.72       503
weighted avg       0.72      0.72      0.72       503



Penjelasan :<br>

1. Confusion Matrix
  * **Random Forest**
    * 194 True Negative (TN) → Model memprediksi kelas negatif, dan memang benar kelas negatif.
    * 59 False Positive (FP) → Model memprediksi kelas positif, tetapi seharusnya kelas negatif.
    * 96 False Negative (FN) → Model memprediksi kelas negatif, tetapi seharusnya kelas positif.
    * 154 True Positive (TP) → Model memprediksi kelas positif, dan memang benar kelas positif.

  * **Logistic Regression**
    * 180 True Negative (TN) → Model memprediksi kelas negatif, dan memang benar kelas negatif.
    * 73 False Positive (FP) → Model memprediksi kelas positif, tetapi seharusnya kelas negatif.
    * 68 False Negative (FN) → Model memprediksi kelas negatif, tetapi seharusnya kelas positif.
    * 182 True Positive (TP) → Model memprediksi kelas positif, dan memang benar kelas positif.


2. Classification Report
     * Kinerja Per Kelas
    * **Random Forest**
        * Random Forest memiliki precision lebih tinggi untuk kelas 1 (0.72) dibanding kelas 0 (0.67), menunjukkan model lebih berhati-hati dalam memprediksi kelas positif.
        * Recall lebih tinggi pada kelas 0 (0.77), yang berarti model lebih baik dalam mendeteksi transaksi yang tidak termasuk dalam kelas positif.
        * False Negative (FN) masih cukup tinggi, menunjukkan bahwa model masih sering salah mengklasifikasikan transaksi positif sebagai negatif.
    * **Logistic Regression**
        * Logistic Regression memiliki keseimbangan antara precision dan recall di kedua kelas (sekitar 0.71-0.73), menunjukkan performa yang lebih stabil dibandingkan Random Forest.
        * False Positive (FP) lebih tinggi dibanding Random Forest (73 vs 59), yang berarti model lebih sering salah memprediksi transaksi sebagai positif.
        * Recall kelas 1 lebih tinggi dari Random Forest (0.73 vs 0.62), yang berarti Logistic Regression lebih baik dalam mengenali transaksi positif.


3. Analisis Model<br>
  * Perbandingan Akurasi Logistic Regression lebih unggul dalam akurasi keseluruhan (72%) dibandingkan Random Forest (69%).
  * Logistic Regression lebih baik dalam mengenali kelas positif, dengan recall 0.73 dibandingkan 0.62 pada Random Forest. rtinya, Logistic Regression lebih jarang salah mengklasifikasikan transaksi positif sebagai negatif.
  * Keseimbangan Prediksi :
    * Logistic Regression lebih seimbang dalam precision dan recall, menghasilkan F1-score yang hampir sama di kedua kelas.
    * Random Forest lebih defensif terhadap kelas positif, sehingga memiliki recall lebih tinggi pada kelas 0 tetapi lebih rendah pada kelas 1.
  * Kesempatan Optimasi :
    * Random Forest dapat ditingkatkan dengan hyperparameter tuning untuk mengurangi False Negative dan meningkatkan recall kelas 1.
    * Logistic Regression dapat diperbaiki dengan balancing data agar mengurangi False Positive.

<br>
**Kesimpulan** <br>
1. Logistic Regression lebih unggul dibandingkan Random Forest dalam akurasi keseluruhan (72% vs 69%).<br>
2. Logistic Regression lebih baik dalam mengenali kelas 1 (Recall = 0.73).<br>
3. Random Forest lebih cocok jika tujuan utama adalah mengurangi False Positives, tetapi perlu ditingkatkan recall pada kelas positif.<br>
4. Perlu dilakukan optimasi lebih lanjut seperti hyperparameter tuning, balancing data, atau mencoba model lain untuk meningkatkan performa klasifikasi.

## **c. Tuning Model Klasifikasi (Optional)**

Tuning dilakukan untuk mencari kombinasi parameter terbaik yang dapat meningkatkan performa model klasifikasi yang masih bisa ditingkatkan dengan mengimplementasikan parameter yang dapat mengurangi kemungkinan underfitting ataupun overfitting. Metode yang digunakan adalah GridSearchCV, yang mencoba berbagai kombinasi parameter dengan validasi silang (cross-validation) untuk menghindari overfitting dan mendapatkan model optimal.

Random Forest

In [13]:
# Hyperparameter tuning Random Forest
rf_params = {
    "n_estimators": [30, 40, 50],  # Kurangi jumlah estimator
    "max_depth": [5, 10, None],  # Batasi kedalaman pohon
    "min_samples_split": [5, 10],  # Minimum sampel untuk split cabang
    "min_samples_leaf": [2, 5]  # Minimum sampel dalam daun
}

# GridSearchCV untuk mencari kombinasi terbaik
rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), rf_params, cv=10, scoring="f1", n_jobs=-1)
rf_grid.fit(X_train, y_train)

# Model terbaik setelah tuning
best_rf_model = rf_grid.best_estimator_

# Evaluasi ulang setelah tuning
rf_tuned_preds = best_rf_model.predict(X_test)
rf_tuned_accuracy = accuracy_score(y_test, rf_tuned_preds)
rf_tuned_f1 = f1_score(y_test, rf_tuned_preds)

# Menampilkan hasil tuning
print("=== Hasil Tuning Random Forest ===")
print(f"Best Params: {rf_grid.best_params_}")
print(f"Testing Accuracy : {rf_tuned_accuracy:.4f}")
print(f"Testing F1-Score : {rf_tuned_f1:.4f}")

=== Hasil Tuning Random Forest ===
Best Params: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 50}
Testing Accuracy : 0.6382
Testing F1-Score : 0.6094


Logistic Regression

In [14]:
# Hyperparameter tuning untuk Logistic Regression
lr_params = {
    "C": [0.01, 0.1, 1, 10, 100],  # Regularisasi
    "solver": ["liblinear", "lbfgs"]  # Solver optimasi
}

# GridSearchCV untuk mencari kombinasi terbaik
lr_grid = GridSearchCV(LogisticRegression(max_iter=200, random_state=42), lr_params, cv=5, scoring="f1", n_jobs=-1)
lr_grid.fit(X_train, y_train)

# Model terbaik setelah tuning
best_lr_model = lr_grid.best_estimator_

# Evaluasi ulang Logistic Regression setelah tuning
lr_tuned_preds = best_lr_model.predict(X_test)
lr_tuned_accuracy = accuracy_score(y_test, lr_tuned_preds)
lr_tuned_f1 = f1_score(y_test, lr_tuned_preds)

# Menampilkan hasil tuning
print("=== Hasil Tuning Logistic Regression ===")
print(f"Best Params: {lr_grid.best_params_}")
print(f"Testing Accuracy : {lr_tuned_accuracy:.4f}")
print(f"Testing F1-Score : {lr_tuned_f1:.4f}")

=== Hasil Tuning Logistic Regression ===
Best Params: {'C': 10, 'solver': 'liblinear'}
Testing Accuracy : 0.9841
Testing F1-Score : 0.9839


In [15]:
# Menyusun hasil evaluasi dalam DataFrame
results_tuning = pd.DataFrame({
    "Model": ["Random Forest (Before)", "Random Forest (After)",
              "Logistic Regression (Before)", "Logistic Regression (After)"],
    "Accuracy": [rf_accuracy, rf_tuned_accuracy, lr_accuracy, lr_tuned_accuracy],
    "F1-Score": [rf_f1, rf_tuned_f1, lr_f1, lr_tuned_f1]
})

# Menampilkan hasil evaluasi setelah tuning
print("=== Hasil Evaluasi Model Sebelum & Sesudah Tuning ===")
print(results_tuning.to_string(index=False))

=== Hasil Evaluasi Model Sebelum & Sesudah Tuning ===
                       Model  Accuracy  F1-Score
      Random Forest (Before)  0.691849  0.665227
       Random Forest (After)  0.638171  0.609442
Logistic Regression (Before)  0.719682  0.720792
 Logistic Regression (After)  0.984095  0.983936


Penjelasan
* Hasil evaluasi menunjukkan bahwa Logistic Regression mengalami peningkatan signifikan setelah tuning, dengan accuracy naik dari 71.96% menjadi 98.40% dan F1-score dari 72.07% menjadi 98.39%, menandakan model menjadi jauh lebih optimal.
* Sebaliknya, Random Forest mengalami penurunan performa setelah tuning, dengan accuracy turun dari 69.18% menjadi 61.83% dan F1-score dari 66.52% menjadi 60.94%, yang mengindikasikan kemungkinan tuning yang kurang optimal atau model menjadi underfitting.
* Kesimpulannya, Logistic Regression adalah model yang lebih baik untuk dataset ini setelah tuning, sementara Random Forest perlu dilakukan tuning ulang agar lebih optimal.

## **d. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.

## Logistic Regression

Perbandingan hasil evaluasi sebelum dan sesudah tuning untuk logistic regression

In [16]:
# 1. Evaluasi sebelum tuning
lr_accuracy_before = lr_accuracy
lr_f1_before = lr_f1
lr_report_before = classification_report(y_test, lr_preds, output_dict=True)

# 2. Evaluasi setelah tuning
lr_accuracy_after = accuracy_score(y_test, lr_tuned_preds)
lr_f1_after = f1_score(y_test, lr_tuned_preds)
lr_report_after = classification_report(y_test, lr_tuned_preds, output_dict=True)

# 3. Menghitung peningkatan performa
acc_improvement = (lr_accuracy_after - lr_accuracy_before) * 100
f1_improvement = (lr_f1_after - lr_f1_before) * 100

In [17]:
# 4. Menampilkan hasil perbandingan sebelum & sesudah tuning
print("=== Perbandingan Performa Logistic Regression ===")
print(f"Accuracy Sebelum Tuning : {lr_accuracy_before:.4f} ({lr_accuracy_before * 100:.2f}%)")
print(f"Accuracy Setelah Tuning : {lr_accuracy_after:.4f} ({lr_accuracy_after * 100:.2f}%)")
print(f"Peningkatan Accuracy    : {acc_improvement:.2f}%\n")

print(f"F1-Score Sebelum Tuning : {lr_f1_before:.4f} ({lr_f1_before * 100:.2f}%)")
print(f"F1-Score Setelah Tuning : {lr_f1_after:.4f} ({lr_f1_after * 100:.2f}%)")
print(f"Peningkatan F1-Score    : {f1_improvement:.2f}%\n")

# 5. Identifikasi kelemahan model berdasarkan Classification Report
print("\n=== Analisis Kelemahan Model ===")
for label in ['0', '1']:
    precision_before = lr_report_before[label]['precision']
    recall_before = lr_report_before[label]['recall']
    precision_after = lr_report_after[label]['precision']
    recall_after = lr_report_after[label]['recall']

    print(f"Kelas {label}:")
    print(f"  - Precision Sebelum: {precision_before:.4f}, Setelah: {precision_after:.4f}")
    print(f"  - Recall Sebelum   : {recall_before:.4f}, Setelah: {recall_after:.4f}")

    if precision_after < 0.8 or recall_after < 0.8:
        print(f"Precision/Recall masih di bawah 0.8 untuk kelas {label}. Bisa jadi ada ketidakseimbangan data.")

# 6. Rekomendasi tindakan lanjutan
print("\n=== Rekomendasi Tindakan Lanjutan ===")
if acc_improvement < 1.0 and f1_improvement < 1.0:
    print("Peningkatan performa kecil. Pertimbangkan metode lain seperti menambah fitur atau balancing data.")
if any(lr_report_after[label]['recall'] < 0.8 for label in ['0', '1']):
    print("Recall masih rendah untuk beberapa kelas. Coba data augmentation atau algoritma lain.")
else :
  print("Model sudah sangat baik. Bisa digunakan untuk deployment.")
# 7. Menampilkan hasil dalam bentuk DataFrame
comparison_df = pd.DataFrame({
    'Metric': ['Accuracy', 'F1-Score'],
    'Before Tuning': [lr_accuracy_before, lr_f1_before],
    'After Tuning': [lr_accuracy_after, lr_f1_after],
    'Improvement (%)': [acc_improvement, f1_improvement]
})



print("\n=== Ringkasan Hasil Evaluasi ===")
print(comparison_df)

=== Perbandingan Performa Logistic Regression ===
Accuracy Sebelum Tuning : 0.7197 (71.97%)
Accuracy Setelah Tuning : 0.9841 (98.41%)
Peningkatan Accuracy    : 26.44%

F1-Score Sebelum Tuning : 0.7208 (72.08%)
F1-Score Setelah Tuning : 0.9839 (98.39%)
Peningkatan F1-Score    : 26.31%


=== Analisis Kelemahan Model ===
Kelas 0:
  - Precision Sebelum: 0.7258, Setelah: 0.9804
  - Recall Sebelum   : 0.7115, Setelah: 0.9881
Kelas 1:
  - Precision Sebelum: 0.7137, Setelah: 0.9879
  - Recall Sebelum   : 0.7280, Setelah: 0.9800

=== Rekomendasi Tindakan Lanjutan ===
Model sudah sangat baik. Bisa digunakan untuk deployment.

=== Ringkasan Hasil Evaluasi ===
     Metric  Before Tuning  After Tuning  Improvement (%)
0  Accuracy       0.719682      0.984095        26.441352
1  F1-Score       0.720792      0.983936        26.314366


Cek Overfitting atau Underfitting

In [18]:
# Evaluasi Logistic Regression di TRAINING SET
lr_train_preds = best_lr_model.predict(X_train)
lr_train_accuracy = accuracy_score(y_train, lr_train_preds)
lr_train_f1 = f1_score(y_train, lr_train_preds)

# Evaluasi Logistic Regression di TESTING SET
lr_test_preds = best_lr_model.predict(X_test)
lr_test_accuracy = accuracy_score(y_test, lr_test_preds)
lr_test_f1 = f1_score(y_test, lr_test_preds)

# Menampilkan hasil evaluasi di kedua set
print("=== Evaluasi Overfitting & Underfitting Model Logistic Regression ===")
print(f"Training Accuracy : {lr_train_accuracy:.4f}")
print(f"Training F1-Score : {lr_train_f1:.4f}")
print(f"Testing Accuracy  : {lr_test_accuracy:.4f}")
print(f"Testing F1-Score  : {lr_test_f1:.4f}")

# Cek perbedaan performa
diff_accuracy = lr_train_accuracy - lr_test_accuracy
diff_f1 = lr_train_f1 - lr_test_f1
print(f"\nPerbedaan Accuracy : {diff_accuracy:.4f}")
print(f"Perbedaan F1-Score : {diff_f1:.4f}")

# Analisis Overfitting atau Underfitting
if diff_accuracy > 0.03 or diff_f1 > 0.03:
    print("\nModel kemungkinan mengalami OVERFITTING. Performa pada training jauh lebih tinggi dibanding testing.")
elif lr_test_accuracy < (lr_train_accuracy * 0.85) or lr_test_f1 < (lr_train_f1 * 0.85):
    print("\nModel kemungkinan mengalami UNDERFITTING. Performa di testing set masih kurang baik.")
else:
    print("\nModel memiliki generalisasi yang baik. Tidak ada indikasi overfitting atau underfitting yang signifikan.")

# Cek Distribusi Prediksi
print("\n=== Distribusi Prediksi ===")
train_pred_counts = pd.Series(lr_train_preds).value_counts(normalize=True) * 100
test_pred_counts = pd.Series(lr_test_preds).value_counts(normalize=True) * 100

print("Distribusi Prediksi di Training Set:")
print(train_pred_counts)
print("\nDistribusi Prediksi di Testing Set:")
print(test_pred_counts)

if any(train_pred_counts < 10) or any(test_pred_counts < 10):
    print("\nModel mungkin mengalami bias kelas! Salah satu kelas diprediksi terlalu sedikit.")

=== Evaluasi Overfitting & Underfitting Model Logistic Regression ===
Training Accuracy : 0.9995
Training F1-Score : 0.9995
Testing Accuracy  : 0.9841
Testing F1-Score  : 0.9839

Perbedaan Accuracy : 0.0154
Perbedaan F1-Score : 0.0156

Model memiliki generalisasi yang baik. Tidak ada indikasi overfitting atau underfitting yang signifikan.

=== Distribusi Prediksi ===
Distribusi Prediksi di Training Set:
0    50.223992
1    49.776008
Name: proportion, dtype: float64

Distribusi Prediksi di Testing Set:
0    50.695825
1    49.304175
Name: proportion, dtype: float64


## Random Forest

Perbandingan hasil evaluasi sebelum dan sesudah tuning untuk Random Forest

In [19]:
# 1. Evaluasi sebelum tuning
rf_accuracy_before = rf_accuracy
rf_f1_before = rf_f1
rf_report_before = classification_report(y_test, rf_preds, output_dict=True)

# 2. Evaluasi setelah tuning
rf_accuracy_after = accuracy_score(y_test, rf_tuned_preds)
rf_f1_after = f1_score(y_test, rf_tuned_preds)
rf_report_after = classification_report(y_test, rf_tuned_preds, output_dict=True)

# 3. Menghitung peningkatan performa
rf_acc_improvement = (rf_accuracy_after - rf_accuracy_before) * 100
rf_f1_improvement = (rf_f1_after - rf_f1_before) * 100

In [20]:
# 4. Menampilkan hasil perbandingan sebelum & sesudah tuning
print("=== Perbandingan Performa Random Forest ===")
print(f"Accuracy Sebelum Tuning : {rf_accuracy_before:.4f} ({rf_accuracy_before * 100:.2f}%)")
print(f"Accuracy Setelah Tuning : {rf_accuracy_after:.4f} ({rf_accuracy_after * 100:.2f}%)")
print(f"Peningkatan Accuracy    : {rf_acc_improvement:.2f}%\n")

print(f"F1-Score Sebelum Tuning : {rf_f1_before:.4f} ({rf_f1_before * 100:.2f}%)")
print(f"F1-Score Setelah Tuning : {rf_f1_after:.4f} ({rf_f1_after * 100:.2f}%)")
print(f"Peningkatan F1-Score    : {rf_f1_improvement:.2f}%\n")

# 5. Identifikasi kelemahan model berdasarkan Classification Report
print("\n=== Analisis Kelemahan Model Random Forest ===")
for label in ['0', '1']:
    precision_before = rf_report_before[label]['precision']
    recall_before = rf_report_before[label]['recall']
    precision_after = rf_report_after[label]['precision']
    recall_after = rf_report_after[label]['recall']

    print(f"Kelas {label}:")
    print(f"  - Precision Sebelum: {precision_before:.4f}, Setelah: {precision_after:.4f}")
    print(f"  - Recall Sebelum   : {recall_before:.4f}, Setelah: {recall_after:.4f}")

    if precision_after < 0.8 or recall_after < 0.8:
        print(f"Precision/Recall masih di bawah 0.8 untuk kelas {label}. Bisa jadi ada ketidakseimbangan data.")

# 6. Rekomendasi tindakan lanjutan
print("\n=== Rekomendasi Tindakan Lanjutan Random Forest ===")
if rf_acc_improvement < 1.0 and rf_f1_improvement < 1.0:
    print("Peningkatan performa kecil. Pertimbangkan metode lain seperti menambah fitur atau balancing data.")
if any(rf_report_after[label]['recall'] < 0.8 for label in ['0', '1']):
    print("Recall masih rendah untuk beberapa kelas. Coba data augmentation atau algoritma lain.")
else:
    print("Model sudah sangat baik. Bisa digunakan untuk deployment.")

# 7. Menampilkan hasil dalam bentuk DataFrame
rf_comparison_df = pd.DataFrame({
    'Metric': ['Accuracy', 'F1-Score'],
    'Before Tuning': [rf_accuracy_before, rf_f1_before],
    'After Tuning': [rf_accuracy_after, rf_f1_after],
    'Improvement (%)': [rf_acc_improvement, rf_f1_improvement]
})

print("\n=== Ringkasan Hasil Evaluasi Random Forest ===")
print(rf_comparison_df)

=== Perbandingan Performa Random Forest ===
Accuracy Sebelum Tuning : 0.6918 (69.18%)
Accuracy Setelah Tuning : 0.6382 (63.82%)
Peningkatan Accuracy    : -5.37%

F1-Score Sebelum Tuning : 0.6652 (66.52%)
F1-Score Setelah Tuning : 0.6094 (60.94%)
Peningkatan F1-Score    : -5.58%


=== Analisis Kelemahan Model Random Forest ===
Kelas 0:
  - Precision Sebelum: 0.6690, Setelah: 0.6237
  - Recall Sebelum   : 0.7668, Setelah: 0.7075
Precision/Recall masih di bawah 0.8 untuk kelas 0. Bisa jadi ada ketidakseimbangan data.
Kelas 1:
  - Precision Sebelum: 0.7230, Setelah: 0.6574
  - Recall Sebelum   : 0.6160, Setelah: 0.5680
Precision/Recall masih di bawah 0.8 untuk kelas 1. Bisa jadi ada ketidakseimbangan data.

=== Rekomendasi Tindakan Lanjutan Random Forest ===
Peningkatan performa kecil. Pertimbangkan metode lain seperti menambah fitur atau balancing data.
Recall masih rendah untuk beberapa kelas. Coba data augmentation atau algoritma lain.

=== Ringkasan Hasil Evaluasi Random Forest ===
   

Cek Overfitting atau Underfitting

In [21]:
# Evaluasi Random Forest di TRAINING SET
rf_train_preds = rf_model.predict(X_train)
rf_train_accuracy = accuracy_score(y_train, rf_train_preds)
rf_train_f1 = f1_score(y_train, rf_train_preds)

# Evaluasi Random Forest di TESTING SET
rf_test_preds = rf_model.predict(X_test)
rf_test_accuracy = accuracy_score(y_test, rf_test_preds)
rf_test_f1 = f1_score(y_test, rf_test_preds)

# Menampilkan hasil evaluasi di kedua set
print("=== Evaluasi Overfitting & Underfitting Model Random Forest ===")
print(f"Training Accuracy : {rf_train_accuracy:.4f}")
print(f"Training F1-Score : {rf_train_f1:.4f}")
print(f"Testing Accuracy  : {rf_test_accuracy:.4f}")
print(f"Testing F1-Score  : {rf_test_f1:.4f}")

# Cek perbedaan performa
diff_accuracy = rf_train_accuracy - rf_test_accuracy
diff_f1 = rf_train_f1 - rf_test_f1
print(f"\nPerbedaan Accuracy : {diff_accuracy:.4f}")
print(f"Perbedaan F1-Score : {diff_f1:.4f}")

# Analisis Overfitting atau Underfitting
if diff_accuracy > 0.03 or diff_f1 > 0.03:
    print("\nModel kemungkinan mengalami OVERFITTING. Performa pada training jauh lebih tinggi dibanding testing.")
elif lr_test_accuracy < (lr_train_accuracy * 0.85) or lr_test_f1 < (lr_train_f1 * 0.85):
    print("\nModel kemungkinan mengalami UNDERFITTING. Performa di testing set masih kurang baik.")
else:
    print("\nModel memiliki generalisasi yang baik. Tidak ada indikasi overfitting atau underfitting yang signifikan.")

# Cek Distribusi Prediksi
print("\n=== Distribusi Prediksi ===")
train_pred_counts = pd.Series(rf_train_preds).value_counts(normalize=True) * 100
test_pred_counts = pd.Series(rf_test_preds).value_counts(normalize=True) * 100

print("Distribusi Prediksi di Training Set:")
print(train_pred_counts)
print("\nDistribusi Prediksi di Testing Set:")
print(test_pred_counts)

if any(train_pred_counts < 10) or any(test_pred_counts < 10):
    print("\nModel mungkin mengalami bias kelas! Salah satu kelas diprediksi terlalu sedikit.")

=== Evaluasi Overfitting & Underfitting Model Random Forest ===
Training Accuracy : 1.0000
Training F1-Score : 1.0000
Testing Accuracy  : 0.6918
Testing F1-Score  : 0.6652

Perbedaan Accuracy : 0.3082
Perbedaan F1-Score : 0.3348

Model kemungkinan mengalami OVERFITTING. Performa pada training jauh lebih tinggi dibanding testing.

=== Distribusi Prediksi ===
Distribusi Prediksi di Training Set:
0    50.273768
1    49.726232
Name: proportion, dtype: float64

Distribusi Prediksi di Testing Set:
0    57.654076
1    42.345924
Name: proportion, dtype: float64


## **KESIMPULAN AKHIR**

1. Performa Model Setelah Tuning

  * Logistic Regression mengalami peningkatan signifikan, dengan:
    - Akurasi meningkat dari 71.97% → 98.41% (+26.44%)
    - F1-score meningkat dari 72.08% → 98.39% (+26.31%)
    
  * Random Forest mengalami penurunan performa setelah tuning:
    - Akurasi menurun dari 69.18% → 63.82% (-5.37%)
    - F1-score menurun dari 66.52% → 60.94% (-5.58%)

2. Evaluasi Overfitting

  * Logistic Regression
    - Training Accuracy: 99.95%, Testing Accuracy: 98.41%
    - Training F1-Score: 99.95%, Testing F1-Score: 98.39%
    - Perbedaan Akurasi: 1.54%, Perbedaan F1-Score: 1.56%
    - Kesimpulan: Model memiliki generalisasi yang sangat baik tanpa indikasi
    - overfitting atau underfitting yang signifikan.

  * Random Forest
    - Training Accuracy: 100.00%, Testing Accuracy: 69.18%
    - Training F1-Score: 100.00%, Testing F1-Score: 66.52%
    - Perbedaan Akurasi: 30.82%, Perbedaan F1-Score: 33.48%
    - Kesimpulan: Model mengalami overfitting karena performa training sangat tinggi (100%) tetapi jauh lebih rendah pada testing.

3. Analisis Kelemahan Model
-Logistic Regression tidak memiliki kelemahan signifikan setelah tuning.
- Random Forest mengalami masalah overfitting dan ketidakseimbangan data: Recall kelas 1 menurun dari 61.60% → 56.80%, menunjukkan model kesulitan mengenali kelas minoritas.

Distribusi prediksi pada testing menunjukkan ketimpangan (kelas 0: 57.65%, kelas 1: 42.35%), yang dapat menyebabkan bias terhadap kelas mayoritas.

4. Upaya Mengatasi Overfitting

  Dalam proses tuning model Random Forest, telah dilakukan beberapa strategi untuk mengurangi kemungkinan overfitting:

  * **Random Forest**
    * Mengurangi jumlah estimator untuk menghindari model yang terlalu kompleks.
    * Membatasi kedalaman pohon keputusan agar tidak menangkap noise dari data.
    * Menyesuaikan minimum sampel untuk split dan leaf agar model lebih generalisasi.
    * Validasi silang (10-fold cross-validation) memastikan model bekerja konsisten di berbagai subset data.
    * Hasilnya, tidak ditemukan indikasi overfitting yang signifikan pada kedua model setelah tuning.
  * **Logistic Regression**
    - Menyesuaikan regulasi (Regularization - C Parameter) agar model tetap sederhana tanpa kehilangan akurasi.
    - Menentukan solver yang optimal (misalnya ‘lbfgs’ atau ‘saga’) untuk menangani dataset dengan fitur kategorikal dan numerikal.
    - Validasi silang (Cross-validation) untuk memastikan model bekerja secara konsisten pada berbagai subset data.

4. Rekomendasi Penggunaan Model<br>
  
  * Logistic Regression dapat digunakan untuk deployment karena:
    - Model lebih sederhana, lebih cepat dalam inferensi, dan tidak rentan terhadap overfitting.
    - Performa sudah sangat tinggi (98.41%) setelah tuning, mendekati model yang lebih kompleks.
    - Cocok untuk kasus klasifikasi biner dengan pola data yang dapat dimodelkan secara linier.<br>
    Jika model diterapkan pada dataset yang berbeda, lakukan penyesuaian parameter atau gunakan teknik seperti balancing data untuk memastikan performa tetap optimal.

  * Random Forest perlu diperbaiki dengan:
    - Balancing Data: Coba SMOTE atau class_weight='balanced' untuk menangani kelas minoritas.
    - Feature Selection: Coba eliminasi fitur yang tidak relevan agar model lebih efisien.
    - Hyperparameter tuning ulang dengan lebih banyak regularisasi untuk mengurangi overfitting.
    - Coba model lain yang bisa menangani ketidakseimbangan data dengan lebih baik.

